# Agent Loan Decisioning

In [104]:
import sys
import datetime
from datetime import date,timedelta
import numpy as np
import pandas as pd
import camelot
import matplotlib.pyplot as plt
import logging

In [105]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logging.info('Code Begin')

2023-05-17 14:43:57,662 - Code Begin


In [106]:
# sys.maxsize
# 2147483647



Qualifying Matrix

1.	Agent Length of Service - 3 months

2.	Agent Frequency of use/activity - >= 150 transactions(count) in total

3.	Agent Frequency of rebalance - >= 50 inbound(count) in total

4.	Median Cash-In for last 90 days - avg. txn amnt - ghs800 

5.	Mode Cash-In for last 90 days - lowest amnt cash in - ghs400 

6.	Median Cash-Out for last 90 days - avg. txn amnt - ghs1000

7.	Mode Cash-Out for last 90 days - lowest amnt cash out - ghs500 

In [107]:

##### full pdf reading

# tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatementReport.pdf',pages='all') #emma data
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\momopages2.pdf',pages='all') # 2 pages prince
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\Momopages3.pdf',pages='all') #50 pages prince
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\lastlast.pdf',pages='all') # 5 pages prince containig each month 
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatement98.pdf',pages='all') # 98 pages prince containig each month 26mi
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatementmax.pdf',pages='all') # maxwell 235 pages 
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatement256.pdf',pages='all') # maxwell 256 pages 
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatement298.pdf',pages='all') # amokoaa regina 298 pages  
tables = camelot.read_pdf(r'C:\Users\Chakdahah\SeiduM.pdf',pages='all') #kuru1 data


tables



# tables[j].parsing_report #check parsing report to see accuracy of parsing and other details

# append to one csv file

table_fin = tables[0].df #convert first table to df and assign to variable 

for i in range(1,len(tables)-1): # skip 0-nth of tables
    print(i)
    t = tables[i].df 
    t.drop(t.index[0], inplace = True) # drop first row of df
    table_fin = pd.concat([table_fin, t], axis = 0)
    

    print(t)


2023-05-17T14:44:06 - INFO - Processing page-1
2023-05-17 14:44:06,462 - Processing page-1
2023-05-17T14:44:13 - INFO - Processing page-2
2023-05-17 14:44:13,067 - Processing page-2
2023-05-17T14:44:19 - INFO - Processing page-3
2023-05-17 14:44:19,480 - Processing page-3
2023-05-17T14:44:24 - INFO - Processing page-4
2023-05-17 14:44:24,937 - Processing page-4


1
                         0         1                          2   \
1   09-Jan-2023 09:49:24 AM  41804111       AbubakarMutari Seidu   
2   09-Jan-2023 06:17:55 AM  41804111       AbubakarMutari Seidu   
3   08-Jan-2023 10:05:38 PM  58141338     DELIBUILD \nTECHNOLOGY   
4   07-Jan-2023 03:55:32 PM  59891814              CHARITY DARKO   
5   06-Jan-2023 04:25:47 PM  41804111       AbubakarMutari Seidu   
6   05-Jan-2023 11:40:45 AM  41804111       AbubakarMutari Seidu   
7   05-Jan-2023 11:38:33 AM  72107788    INTEROPERABILITY \nPULL   
8   05-Jan-2023 10:35:08 AM  41804111       AbubakarMutari Seidu   
9   05-Jan-2023 10:18:49 AM  45842117   JUDOS COMPANY  \nLIMITED   
10  02-Jan-2023 09:15:16 AM  41804111       AbubakarMutari Seidu   
11  31-Dec-2022 03:35:44 PM  54848810  VENTURES GATORWU \nEUNICE   
12  26-Dec-2022 06:12:50 AM  41804111       AbubakarMutari Seidu   
13  25-Dec-2022 11:50:14 AM  41804111       AbubakarMutari Seidu   
14  24-Dec-2022 12:49:51 PM  41804111       Ab

In [108]:
# Assign tables into momo data variable

momo_data = table_fin
momo_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,TRANSACTION DATE,FROM ACCT,FROM NAME,FROM NO.,TRANS. TYPE,AMOUNT,FEES,E-LEVY,BAL BEFORE,BAL AFTER,TO NO.,TO NAME,TO ACCT,F_ID,REF,OVA
1,20-Jan-2023 11:09:13 AM,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,170,1.7,0,180.13,8.43,233554866967,JUDOS COMPANY \nLIMITED,45842117,21589388156,NationalId--,Internal
2,20-Jan-2023 11:05:46 AM,59747276,AHOMKA-PERSONAL,0,TRANSFER,177,0,0,3.13,180.13,233243766898,AbubakarMutari \nSeidu,41804111,21589322868,,Internal
3,20-Jan-2023 10:33:00 AM,41804111,AbubakarMutari Seidu,233243766898,DEBIT,53.45,0,0,56.58,3.13,0,OPSWALLET-\nPERSONAL,58138232,21588686876,,OPSWALLET-PERSONAL
4,20-Jan-2023 10:30:51 AM,45842117,JUDOS COMPANY \nLIMITED,233554866967,CASH_IN,50,0,0,6.58,56.58,233243766898,AbubakarMutari \nSeidu,41804111,21588644584,Cashin To-\n233243766898,Internal
5,16-Jan-2023 07:34:15 PM,41804111,,233243766898,CASH_OUT,50,0.5,0,57.08,6.58,233558612087,,56545905,21523347982,NationalId--,Internal
6,16-Jan-2023 07:02:07 PM,58138232,,0,TRANSFER,50,0,0,7.08,57.08,233243766898,,41804111,21522654697,,Internal
7,15-Jan-2023 01:46:20 PM,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,10.08,7.08,0,MTNONLINEAIRTI\nMEVENDOR,39011161,21498071639,,MTNONLINEAIRTIMEVEND\nOR
8,12-Jan-2023 02:24:47 PM,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,22,0.5,0,32.58,10.08,233554866967,JUDOS COMPANY \nLIMITED,45842117,21443534730,NationalId--,Internal
9,12-Jan-2023 01:54:04 PM,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,100,0.75,0,133.33,32.58,233598605162,TIGO PUSH OVA,71396410,21442994330,"MOHAMMED \nBASHIRU \nZAKARI,23327576\n5697,1",mmipushtigo


In [109]:
momo_data[~momo_data.iloc[:,0].str.contains("\n")]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,TRANSACTION DATE,FROM ACCT,FROM NAME,FROM NO.,TRANS. TYPE,AMOUNT,FEES,E-LEVY,BAL BEFORE,BAL AFTER,TO NO.,TO NAME,TO ACCT,F_ID,REF,OVA
1,20-Jan-2023 11:09:13 AM,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,170,1.7,0,180.13,8.43,233554866967,JUDOS COMPANY \nLIMITED,45842117,21589388156,NationalId--,Internal
2,20-Jan-2023 11:05:46 AM,59747276,AHOMKA-PERSONAL,0,TRANSFER,177,0,0,3.13,180.13,233243766898,AbubakarMutari \nSeidu,41804111,21589322868,,Internal
3,20-Jan-2023 10:33:00 AM,41804111,AbubakarMutari Seidu,233243766898,DEBIT,53.45,0,0,56.58,3.13,0,OPSWALLET-\nPERSONAL,58138232,21588686876,,OPSWALLET-PERSONAL
4,20-Jan-2023 10:30:51 AM,45842117,JUDOS COMPANY \nLIMITED,233554866967,CASH_IN,50,0,0,6.58,56.58,233243766898,AbubakarMutari \nSeidu,41804111,21588644584,Cashin To-\n233243766898,Internal
5,16-Jan-2023 07:34:15 PM,41804111,,233243766898,CASH_OUT,50,0.5,0,57.08,6.58,233558612087,,56545905,21523347982,NationalId--,Internal
6,16-Jan-2023 07:02:07 PM,58138232,,0,TRANSFER,50,0,0,7.08,57.08,233243766898,,41804111,21522654697,,Internal
7,15-Jan-2023 01:46:20 PM,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,10.08,7.08,0,MTNONLINEAIRTI\nMEVENDOR,39011161,21498071639,,MTNONLINEAIRTIMEVEND\nOR
8,12-Jan-2023 02:24:47 PM,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,22,0.5,0,32.58,10.08,233554866967,JUDOS COMPANY \nLIMITED,45842117,21443534730,NationalId--,Internal
9,12-Jan-2023 01:54:04 PM,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,100,0.75,0,133.33,32.58,233598605162,TIGO PUSH OVA,71396410,21442994330,"MOHAMMED \nBASHIRU \nZAKARI,23327576\n5697,1",mmipushtigo


In [110]:
momo_data.columns = momo_data.iloc[0] #set column names equal to values in row index position 0
momo_data = momo_data[1:] # remove first row from DataFrame

In [111]:
momo_data.reset_index(drop=True, inplace=True)

In [112]:
momo_data

,TRANSACTION DATE,FROM ACCT,FROM NAME,FROM NO.,TRANS. TYPE,AMOUNT,FEES,E-LEVY,BAL BEFORE,BAL AFTER,TO NO.,TO NAME,TO ACCT,F_ID,REF,OVA
0,20-Jan-2023 11:09:13 AM,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,170,1.7,0,180.13,8.43,233554866967,JUDOS COMPANY \nLIMITED,45842117,21589388156,NationalId--,Internal
1,20-Jan-2023 11:05:46 AM,59747276,AHOMKA-PERSONAL,0,TRANSFER,177,0,0,3.13,180.13,233243766898,AbubakarMutari \nSeidu,41804111,21589322868,,Internal
2,20-Jan-2023 10:33:00 AM,41804111,AbubakarMutari Seidu,233243766898,DEBIT,53.45,0,0,56.58,3.13,0,OPSWALLET-\nPERSONAL,58138232,21588686876,,OPSWALLET-PERSONAL
3,20-Jan-2023 10:30:51 AM,45842117,JUDOS COMPANY \nLIMITED,233554866967,CASH_IN,50,0,0,6.58,56.58,233243766898,AbubakarMutari \nSeidu,41804111,21588644584,Cashin To-\n233243766898,Internal
4,16-Jan-2023 07:34:15 PM,41804111,,233243766898,CASH_OUT,50,0.5,0,57.08,6.58,233558612087,,56545905,21523347982,NationalId--,Internal
5,16-Jan-2023 07:02:07 PM,58138232,,0,TRANSFER,50,0,0,7.08,57.08,233243766898,,41804111,21522654697,,Internal
6,15-Jan-2023 01:46:20 PM,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,10.08,7.08,0,MTNONLINEAIRTI\nMEVENDOR,39011161,21498071639,,MTNONLINEAIRTIMEVEND\nOR
7,12-Jan-2023 02:24:47 PM,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,22,0.5,0,32.58,10.08,233554866967,JUDOS COMPANY \nLIMITED,45842117,21443534730,NationalId--,Internal
8,12-Jan-2023 01:54:04 PM,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,100,0.75,0,133.33,32.58,233598605162,TIGO PUSH OVA,71396410,21442994330,"MOHAMMED \nBASHIRU \nZAKARI,23327576\n5697,1",mmipushtigo
9,12-Jan-2023 11:26:08 AM,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,136.33,133.33,0,MTNONLINEAIRTI\nMEVENDOR,39011161,21440421699,,MTNONLINEAIRTIMEVEND\nOR


In [113]:
#Reading data from csv file

# momo_data = pd.read_csv(r'\Users\Chakdahah\Cleaned_Momo.csv', skipinitialspace = True)

# Clean whitespaces in colunn name and entire dataframe

momo_data =momo_data.rename(columns=lambda x: x.strip())
momo_data =momo_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

momo_data.head()

,TRANSACTION DATE,FROM ACCT,FROM NAME,FROM NO.,TRANS. TYPE,AMOUNT,FEES,E-LEVY,BAL BEFORE,BAL AFTER,TO NO.,TO NAME,TO ACCT,F_ID,REF,OVA
0,20-Jan-2023 11:09:13 AM,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,170,1.7,0,180.13,8.43,233554866967,JUDOS COMPANY \nLIMITED,45842117,21589388156,NationalId--,Internal
1,20-Jan-2023 11:05:46 AM,59747276,AHOMKA-PERSONAL,0,TRANSFER,177,0,0,3.13,180.13,233243766898,AbubakarMutari \nSeidu,41804111,21589322868,,Internal
2,20-Jan-2023 10:33:00 AM,41804111,AbubakarMutari Seidu,233243766898,DEBIT,53.45,0,0,56.58,3.13,0,OPSWALLET-\nPERSONAL,58138232,21588686876,,OPSWALLET-PERSONAL
3,20-Jan-2023 10:30:51 AM,45842117,JUDOS COMPANY \nLIMITED,233554866967,CASH_IN,50,0,0,6.58,56.58,233243766898,AbubakarMutari \nSeidu,41804111,21588644584,Cashin To-\n233243766898,Internal
4,16-Jan-2023 07:34:15 PM,41804111,,233243766898,CASH_OUT,50,0.5,0,57.08,6.58,233558612087,,56545905,21523347982,NationalId--,Internal


In [114]:

# remove unwanted characters from end of column names
# mmomo_data.rename(columns=lambda x: x.strip('.'),inplace =True)

# remove all unwanted char acters in column names
momo_data.rename(columns=lambda s: s.replace(".", ""), inplace=True)

# replace whitespaces with underscore
momo_data.rename(columns=lambda s: s.replace(" ", "_"), inplace=True)

In [115]:
# cast appropriate data types
momo_data['AMOUNT'] = pd.to_numeric(momo_data['AMOUNT'],errors='coerce')
momo_data['BAL_BEFORE'] = pd.to_numeric(momo_data['BAL_BEFORE'],errors='coerce')
momo_data['BAL_AFTER'] = pd.to_numeric(momo_data['BAL_AFTER'],errors='coerce')
momo_data['F_ID'] = pd.to_numeric(momo_data['F_ID'],errors='coerce')


In [116]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TRANSACTION_DATE  52 non-null     object 
 1   FROM_ACCT         52 non-null     object 
 2   FROM_NAME         52 non-null     object 
 3   FROM_NO           52 non-null     object 
 4   TRANS_TYPE        52 non-null     object 
 5   AMOUNT            52 non-null     float64
 6   FEES              52 non-null     object 
 7   E-LEVY            52 non-null     object 
 8   BAL_BEFORE        52 non-null     float64
 9   BAL_AFTER         52 non-null     float64
 10  TO_NO             52 non-null     object 
 11  TO_NAME           52 non-null     object 
 12  TO_ACCT           52 non-null     object 
 13  F_ID              52 non-null     int64  
 14  REF               52 non-null     object 
 15  OVA               52 non-null     object 
dtypes: float64(3), int64(1), object(12)
memory usa

In [117]:
#Suppressing any scientific notation in entire csv file.

pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [118]:
momo_data.head()

,TRANSACTION_DATE,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,E-LEVY,BAL_BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
0,20-Jan-2023 11:09:13 AM,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,170,1.7,0,180,8,233554866967,JUDOS COMPANY \nLIMITED,45842117,21589388156,NationalId--,Internal
1,20-Jan-2023 11:05:46 AM,59747276,AHOMKA-PERSONAL,0,TRANSFER,177,0,0,3,180,233243766898,AbubakarMutari \nSeidu,41804111,21589322868,,Internal
2,20-Jan-2023 10:33:00 AM,41804111,AbubakarMutari Seidu,233243766898,DEBIT,53,0,0,57,3,0,OPSWALLET-\nPERSONAL,58138232,21588686876,,OPSWALLET-PERSONAL
3,20-Jan-2023 10:30:51 AM,45842117,JUDOS COMPANY \nLIMITED,233554866967,CASH_IN,50,0,0,7,57,233243766898,AbubakarMutari \nSeidu,41804111,21588644584,Cashin To-\n233243766898,Internal
4,16-Jan-2023 07:34:15 PM,41804111,,233243766898,CASH_OUT,50,0.5,0,57,7,233558612087,,56545905,21523347982,NationalId--,Internal


In [119]:
m =momo_data['TRANSACTION_DATE'].str.split(' ', expand=True)


In [120]:

momo_data['TRANSACTION_DATE'] = m[0]
momo_data['TRANSACTION_TIME'] = m[1]


In [121]:
# shift column 'C' to first position
first_column = momo_data.pop('TRANSACTION_TIME')
  
# insert column using insert(position,column_name,first_column) function
momo_data.insert(1, 'TRANSACTION_TIME', first_column)
  

In [122]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TRANSACTION_DATE  52 non-null     object 
 1   TRANSACTION_TIME  52 non-null     object 
 2   FROM_ACCT         52 non-null     object 
 3   FROM_NAME         52 non-null     object 
 4   FROM_NO           52 non-null     object 
 5   TRANS_TYPE        52 non-null     object 
 6   AMOUNT            52 non-null     float64
 7   FEES              52 non-null     object 
 8   E-LEVY            52 non-null     object 
 9   BAL_BEFORE        52 non-null     float64
 10  BAL_AFTER         52 non-null     float64
 11  TO_NO             52 non-null     object 
 12  TO_NAME           52 non-null     object 
 13  TO_ACCT           52 non-null     object 
 14  F_ID              52 non-null     int64  
 15  REF               52 non-null     object 
 16  OVA               52 non-null     object 
dtyp

In [123]:
momo_data['FROM_NAME'] = momo_data.FROM_NAME.str.replace('\n',' ')

In [124]:
momo_data.tail()

,TRANSACTION_DATE,TRANSACTION_TIME,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,E-LEVY,BAL_BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
47,27-Nov-2022,05:08:26,41804111,AbubakarMutari Seidu,233243766898,DEBIT,3,0,0,43,40,0,cisnew,62525623,20586088567,Internet Bundle,cisnew
48,27-Nov-2022,04:39:56,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,46,43,0,MTNONLINEAIRTI\nMEVENDOR,39011161,20585554424,,MTNONLINEAIRTIMEVEND\nOR
49,25-Nov-2022,09:18:46,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,49,46,0,MTNONLINEAIRTI\nMEVENDOR,39011161,20553872842,,MTNONLINEAIRTIMEVEND\nOR
50,24-Nov-2022,09:08:02,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,52,49,0,MTNONLINEAIRTI\nMEVENDOR,39011161,20519275619,,MTNONLINEAIRTIMEVEND\nOR
51,21-Nov-2022,08:39:54,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,55,52,0,MTNONLINEAIRTI\nMEVENDOR,39011161,20456480245,,MTNONLINEAIRTIMEVEND\nOR


In [125]:
num=momo_data.loc[momo_data['TRANS_TYPE'] == 'CASH_IN', 'FROM_NO'].iloc[0]
num

'233554866967'

In [126]:
momo_data

,TRANSACTION_DATE,TRANSACTION_TIME,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,E-LEVY,BAL_BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
0,20-Jan-2023,11:09:13,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,170,1.7,0,180,8,233554866967,JUDOS COMPANY \nLIMITED,45842117,21589388156,NationalId--,Internal
1,20-Jan-2023,11:05:46,59747276,AHOMKA-PERSONAL,0,TRANSFER,177,0,0,3,180,233243766898,AbubakarMutari \nSeidu,41804111,21589322868,,Internal
2,20-Jan-2023,10:33:00,41804111,AbubakarMutari Seidu,233243766898,DEBIT,53,0,0,57,3,0,OPSWALLET-\nPERSONAL,58138232,21588686876,,OPSWALLET-PERSONAL
3,20-Jan-2023,10:30:51,45842117,JUDOS COMPANY LIMITED,233554866967,CASH_IN,50,0,0,7,57,233243766898,AbubakarMutari \nSeidu,41804111,21588644584,Cashin To-\n233243766898,Internal
4,16-Jan-2023,07:34:15,41804111,,233243766898,CASH_OUT,50,0.5,0,57,7,233558612087,,56545905,21523347982,NationalId--,Internal
5,16-Jan-2023,07:02:07,58138232,,0,TRANSFER,50,0,0,7,57,233243766898,,41804111,21522654697,,Internal
6,15-Jan-2023,01:46:20,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,10,7,0,MTNONLINEAIRTI\nMEVENDOR,39011161,21498071639,,MTNONLINEAIRTIMEVEND\nOR
7,12-Jan-2023,02:24:47,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,22,0.5,0,33,10,233554866967,JUDOS COMPANY \nLIMITED,45842117,21443534730,NationalId--,Internal
8,12-Jan-2023,01:54:04,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,100,0.75,0,133,33,233598605162,TIGO PUSH OVA,71396410,21442994330,"MOHAMMED \nBASHIRU \nZAKARI,23327576\n5697,1",mmipushtigo
9,12-Jan-2023,11:26:08,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,136,133,0,MTNONLINEAIRTI\nMEVENDOR,39011161,21440421699,,MTNONLINEAIRTIMEVEND\nOR


In [160]:
momo = momo_data.loc[momo_data['OVA'].str.contains("cis", case = False)]
momo

,TRANSACTION_DATE,TRANSACTION_TIME,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,E-LEVY,BAL_BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
15,2023-01-09,06:17:55,41804111,AbubakarMutari Seidu,233243766898,DEBIT,3,0,0,2375,2372,0,cisnew,62525623,21380084865,Internet Bundle,cisnew
29,2022-12-18,10:03:30,41804111,AbubakarMutari Seidu,233243766898,DEBIT,3,0,0,11,8,0,cisnew,62525623,20982479840,Internet Bundle,cisnew
44,2022-12-02,09:59:25,41804111,AbubakarMutari Seidu,233243766898,DEBIT,3,0,0,34,31,0,cisnew,62525623,20679471996,Internet Bundle,cisnew
47,2022-11-27,05:08:26,41804111,AbubakarMutari Seidu,233243766898,DEBIT,3,0,0,43,40,0,cisnew,62525623,20586088567,Internet Bundle,cisnew


In [161]:
name = momo['FROM_NAME'].iloc[0]
name

'AbubakarMutari Seidu'

In [130]:
# momo_data[~momo_data.iloc[:,0].str.contains("\n")]

,TRANSACTION_DATE,TRANSACTION_TIME,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,E-LEVY,BAL_BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
0,20-Jan-2023,11:09:13,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,170,1.7,0,180,8,233554866967,JUDOS COMPANY \nLIMITED,45842117,21589388156,NationalId--,Internal
1,20-Jan-2023,11:05:46,59747276,AHOMKA-PERSONAL,0,TRANSFER,177,0,0,3,180,233243766898,AbubakarMutari \nSeidu,41804111,21589322868,,Internal
2,20-Jan-2023,10:33:00,41804111,AbubakarMutari Seidu,233243766898,DEBIT,53,0,0,57,3,0,OPSWALLET-\nPERSONAL,58138232,21588686876,,OPSWALLET-PERSONAL
3,20-Jan-2023,10:30:51,45842117,JUDOS COMPANY LIMITED,233554866967,CASH_IN,50,0,0,7,57,233243766898,AbubakarMutari \nSeidu,41804111,21588644584,Cashin To-\n233243766898,Internal
4,16-Jan-2023,07:34:15,41804111,,233243766898,CASH_OUT,50,0.5,0,57,7,233558612087,,56545905,21523347982,NationalId--,Internal
5,16-Jan-2023,07:02:07,58138232,,0,TRANSFER,50,0,0,7,57,233243766898,,41804111,21522654697,,Internal
6,15-Jan-2023,01:46:20,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,10,7,0,MTNONLINEAIRTI\nMEVENDOR,39011161,21498071639,,MTNONLINEAIRTIMEVEND\nOR
7,12-Jan-2023,02:24:47,41804111,AbubakarMutari Seidu,233243766898,CASH_OUT,22,0.5,0,33,10,233554866967,JUDOS COMPANY \nLIMITED,45842117,21443534730,NationalId--,Internal
8,12-Jan-2023,01:54:04,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,100,0.75,0,133,33,233598605162,TIGO PUSH OVA,71396410,21442994330,"MOHAMMED \nBASHIRU \nZAKARI,23327576\n5697,1",mmipushtigo
9,12-Jan-2023,11:26:08,41804111,AbubakarMutari Seidu,233243766898,PAYMENT,3,0,0,136,133,0,MTNONLINEAIRTI\nMEVENDOR,39011161,21440421699,,MTNONLINEAIRTIMEVEND\nOR


In [132]:
# momo_data =momo_data.dropna()

In [133]:
# store date as object for later use
mi = momo_data['TRANSACTION_DATE']

In [134]:
momo_data['TRANSACTION_DATE'] = pd.to_datetime(momo_data['TRANSACTION_DATE'])

# Converting PDF to CSV and XLSX


In [135]:
## convert momo_data to csv
momo_data.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\StrippedMomotoCSVseidu.csv',date_format='%d/%m/%Y')

momo_data.to_excel("StrippedMomotoXLSseidu.xlsx")

In [136]:
# Set empty list and dict
masterData = []
userData= {}

# Calculation of scoring parameters

In [137]:

rows = momo_data.count()[0] # Agent Frequency of use
userData["freq_of_active"] = rows



MOMO=momo_data[(momo_data == 'CASH_IN').any(axis=1)] # frequency of rebalance
MOMOR = MOMO.count()[0]
userData["freq_of_repl"] = MOMOR


meadnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].median() #Median Cash-In for last 90days
# userData["mead_cash_in"] = meadnie
# userData["mead_cash_in"]
userData["median_cash_in_calc"] = meadnie*4


modnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].mode() # Mode Cash-In for last 90 days
userData["mode_cash_in_calc"] = modnie*4
userData["mode_cash_in_calc"] = max(userData["mode_cash_in_calc"]) #for converting from bool to int for processing


meadniout = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].median() # Median Cash-Out for last 90 days
# userData["mead_cash_out"] = meadniout
# userData["mead_cash_out"]
userData["median_cash_out_calc"] = meadniout*4


modno = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].mode() # Mode Cash-Out for last 90 days
userData["mode_cash_out_calc"] = modno*4 
userData["mode_cash_out_calc"] = max(userData["mode_cash_out_calc"])




# Scoring Criteria Calculation

In [141]:
user_copy = userData.copy()
masterData.append(user_copy)

In [142]:
### scoring

inelligible = []
elligible = []
score_table = []
scoring_table = {}
combined = []

In [143]:
scoring_table["Agent Number"] = num
scoring_table["Agent Name"] = name

In [144]:


## Agent Length of Service - 3 months

for i in mi:
    u = mi.str.extract('([a-zA-Z ]+)', expand=False).str.strip()
    monthcount = u.drop_duplicates()
print(monthcount)

if len(monthcount)>=3:
    scoring_table["Length of Service"] = 100
    Credit_score = 100
else:
    scoring_table["Length of Service"] = 0
    Credit_score = 0


for index in range(len(masterData)):

    ## Frequency of use  >= 150 transactions(count) in total

    if masterData[index]["freq_of_active"]> 150 and masterData[index]["freq_of_active"] <= 300:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50
        scoring_table["Frequency of Activity"] = 50
    elif masterData[index]["freq_of_active"]> 300:
        elig_copy = masterData[index]
        Credit_score+= 100
        scoring_table["Frequency of Activity"] = 100
    else:
        scoring_table["Frequency of Activity"] = 0


        

        
        
#Agent Frequency of rebalance - >= 50 inbound(count) in total

    if masterData[index]["freq_of_repl"]>= 50 and masterData[index]["freq_of_repl"] <= 100:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Frequency of Rebalance"] = 50
    elif masterData[index]["freq_of_active"]> 100:
        elig_copy = masterData[index]
        Credit_score+= 100
        scoring_table["Frequency of Rebalance"] = 100
    else:
        scoring_table["Frequency of Rebalance"] = 0
        

        
        
        # Median Cash-In for last 90 days - avg. txn amnt - ghs800 

        
    if masterData[index]["median_cash_in_calc"] >= 800:
        user_copy = masterData[index]
        Credit_score += 50
        scoring_table["Median Cash In"] = 50
    else:
        Credit_score+= 0
        scoring_table["Median Cash In"] = 0

        
        
          # Mode Cash-In for last 90 days - lowest amnt cash in - ghs400 

        
    if masterData[index]["mode_cash_in_calc"] >= 400:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Mode Cash In"] = 50
        
    else:
        elig_copy = masterData[index]
        Credit_score+= 0
        scoring_table["Mode Cash In"] = 0
        
        
        
       # Median Cash-Out for last 90 days - avg. txn amnt - ghs1000


    if masterData[index]["median_cash_out_calc"] > 1000:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Median Cash Out"] = 50
    else:
        elig_copy = masterData[index]
        Credit_score+= 0
        scoring_table["Median Cash Out"] = 0
        
#     Mode Cash-Out for last 90 days - lowest amnt cash out - ghs500 
        
    if masterData[index]["mode_cash_out_calc"] >= 500:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Mode Cash Out"] = 50
    else:
        elig_copy = masterData[index]
        Credit_score+= 0   
        scoring_table["Mode Cash Out"] = 0
    

#         print(elig_copy)
scoring_table["Credit Score"] = Credit_score
score_table.append(scoring_table)        
        
    

0     Jan
24    Dec
45    Nov
Name: TRANSACTION_DATE, dtype: object


In [145]:
elligible.append(elig_copy)

# Applying scores to Merchants

In [146]:
    
    if Credit_score < 200:
        print("You don't qualify for a loan")
        scoring_table["Eligibility"] = 0
    
    elif Credit_score >= 200 and Credit_score <= 250:
        print("You qualify for a GHS 1000 loan")
        scoring_table["Eligibility"] = 1000

    elif Credit_score >= 251 and Credit_score <= 350:
        print("You qualify for a GHS 2000 loan")
        scoring_table["Eligibility"] = 2000

    elif Credit_score >= 351 and Credit_score <= 400:
        print("You qualify for a GHS 3000 loan")
        scoring_table["Eligibility"] = 3000


    elif Credit_score >= 401 and Credit_score <= 500:
        print("You qualify for a GHS 5000 loan")
        scoring_table["Eligibility"] = 5000



# Combining Eligibility and Score Tables

In [147]:
comb = scoring_table | elig_copy
comb


{'Agent Number': '233554866967',
 'Agent Name': 'AbubakarMutari Seidu',
 'Length of Service': 100,
 'Frequency of Activity': 0,
 'Frequency of Rebalance': 0,
 'Median Cash In': 0,
 'Mode Cash In': 0,
 'Median Cash Out': 0,
 'Mode Cash Out': 0,
 'Credit Score': 100,
 'freq_of_active': 52,
 'freq_of_repl': 7,
 'median_cash_in_calc': 420.0,
 'mode_cash_in_calc': 688.0,
 'median_cash_out_calc': 560.0,
 'mode_cash_out_calc': 560.0}

In [148]:
combined.append(comb)

In [149]:
print("Elligible", elligible)

Elligible [{'freq_of_active': 52, 'freq_of_repl': 7, 'median_cash_in_calc': 420.0, 'mode_cash_in_calc': 688.0, 'median_cash_out_calc': 560.0, 'mode_cash_out_calc': 560.0}]


In [150]:
Credit_score

100

# Converting and saving Score Table

In [151]:
score_table = pd.DataFrame.from_dict(score_table)
score_table

,Agent Number,Agent Name,Length of Service,Frequency of Activity,Frequency of Rebalance,Median Cash In,Mode Cash In,Median Cash Out,Mode Cash Out,Credit Score
0,233554866967,AbubakarMutari Seidu,100,0,0,0,0,0,0,100


In [152]:
score_table.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\scoretableseidu.csv')

# Converting and saving Eligibility Table

In [153]:
### convert dict to df

elligible = pd.DataFrame.from_dict(elligible)
elligible

,freq_of_active,freq_of_repl,median_cash_in_calc,mode_cash_in_calc,median_cash_out_calc,mode_cash_out_calc
0,52,7,420,688,560,560


In [154]:
type(elligible)

pandas.core.frame.DataFrame

In [155]:
elligible.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\elligibleseidu.csv')

# Converting and saving Combined Table

In [156]:
combined_table = pd.DataFrame.from_dict(combined)
combined_table

,Agent Number,Agent Name,Length of Service,Frequency of Activity,Frequency of Rebalance,Median Cash In,Mode Cash In,Median Cash Out,Mode Cash Out,Credit Score,freq_of_active,freq_of_repl,median_cash_in_calc,mode_cash_in_calc,median_cash_out_calc,mode_cash_out_calc
0,233554866967,AbubakarMutari Seidu,100,0,0,0,0,0,0,100,52,7,420,688,560,560


In [157]:
combined_table.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\combtableseidu.csv')

In [158]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logging.info('End of Code')

2023-05-17 14:44:31,085 - End of Code


# Score and risk grade table 

In [159]:
details = {
    'Risk Ranking' : ['RR-1', 'RR-2', 'RR-3', 'RR-4'],
    'Minimum Score' : [200,251,351,401],
    'Maximum Score' : [250,350,400,500],
    'Qualifying Amount(GHS)':[1000,2000,3000,5000]
}
  
# creating a Dataframe object 
df = pd.DataFrame(details)
df

,Risk Ranking,Minimum Score,Maximum Score,Qualifying Amount(GHS)
0,RR-1,200,250,1000
1,RR-2,251,350,2000
2,RR-3,351,400,3000
3,RR-4,401,500,5000
